In [1]:
from selenium import webdriver
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import ast
import json
import time 

In [2]:
class WebScraping:
    website_url="https://www.expresslanes.com/map-your-trip"
    data_point={}
    odds_point={}
    def __init__(self,url):
        if(url!=self.website_url):
            raise Exception("this url is not supported")
        self.website_url=url
        
    def getEntryAndExitData(self):
        driver = webdriver.Chrome('C:/Users/abhis/Downloads/chromedriver')
        driver.get(self.website_url)
        time.sleep(10)
        if driver.find_element_by_id("EntrySelect").text=='':
            raise Exception("you didn't chose any specify direction")
        content=[]
        if len(driver.find_elements_by_css_selector("option[data-direction='Southbound']"))!=1:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            content = soup.find_all("option", {"data-direction": "Southbound"})
        else:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            content = soup.find_all("option", {"data-direction": "Northbound"})
        
        for i in content:
            if(i.attrs.get("data-bubbleid")==None):
                continue
            self.data_point[i.attrs.get("data-bubbleid")[:-2]]=i.text
        for i in content:
            if(i.attrs.get("data-bubbleid")==None):
                continue
            temp_list=ast.literal_eval(i.attrs.get('data-exitids'))
            for j in temp_list:
                for k in j["ods"]:
                    if(j["id"][:-2] in self.data_point):
                        self.odds_point[k]=[i.attrs.get("value"),j["id"]]
    
    def getApiData(self):
        api_url="https://www.expresslanes.com/maps-api/infra-price-confirmed-all"
        r=requests.get(api_url)
        if r.status_code!=200:
            raise Exception("Api is not reachable")
        url_data=r.json()
        return url_data
    def generateCsvFile(self,url_data):
        result=pd.DataFrame(columns=["entry_id","exit_id","entry_label","exit_label","ods","path","Direction","status","price","time"])
        for points in url_data["response"]:
            if points["od"][3:] in self.odds_point:
                temp_data={"entry_id":self.odds_point[points["od"][3:]][0],"exit_id":self.odds_point[points["od"][3:]][1],"entry_label":self.data_point[self.odds_point[points["od"][3:]][0][:-2]],"exit_label":self.data_point[self.odds_point[points["od"][3:]][1][:-2]],"ods":points["od"],"path":points["road"],"Direction":points["direction"],"status":points["status"],"price":points["price"],"time":points["time"]}
                result=result.append(temp_data,ignore_index=True)

        result.to_csv("result.csv",index=False)
    def main(self):
        self.getEntryAndExitData()
        url_data=self.getApiData()
        self.generateCsvFile(url_data)

In [5]:
object1=WebScraping("https://www.expresslanes.com/map-your-trip")

In [6]:
object1.main()